# Train Abstracts BERT

From the all abstracts corpus, we only select the texts associated to CIE-D codes present in train CodiEsp corpus, generating the train-asbtracts corpus.

In [1]:
import pandas as pd
import numpy as np

In [2]:
corpus_path = "../datasets/abstractsWithCIE10_v2/"

## Load pre-processed text

Firstly, text from all additional abstracts with CIE-D codes associated are loaded. We use the pre-processed version of the corpus where only texts with BETO sub-token length >= 30 are contained (see `BERT-Keras-All-Abstracts` notebook):

In [3]:
%%time
# Pre-processed version (BETO len >= 30)
df_text_all_abs = pd.read_table(corpus_path + "all_abstracts_valid_codes_D_text_raw_sw_v2_30.tsv", sep='\t')

CPU times: user 1.93 s, sys: 184 ms, total: 2.11 s
Wall time: 2.11 s


In [4]:
df_text_all_abs.shape

(149424, 3)

In [5]:
df_text_all_abs.head()

doc_id                                           raw_text  \
0  biblio-1000005  Introducción: A pesar del difícil acceso anató...   
1  biblio-1000026  Introducción: La enterocolitis neutropénica se...   
2  biblio-1000027  Introducción: La presencia de anticuerpos anti...   
3  biblio-1000028  Introducción: El Carcinoma de lengua móvil es ...   
4  biblio-1000029  Introducción: El cáncer de ovario epitelial au...   

                                             sw_text  
0  Introducción : pesar difícil acceso anatómico ...  
1  Introducción : enterocolitis neutropénica defi...  
2  Introducción : presencia anticuerpos anti erit...  
3  Introducción : Carcinoma lengua móvil tumores ...  
4  Introducción : cáncer ovario epitelial aunque ...

In [6]:
df_text_all_abs.raw_text[0]

'Introducción: A pesar del difícil acceso anatómico para los tumores de mediastino, la resección quirúrgica sigue siendo el mejor enfoque diagnóstico y terapéutico. El objetivo de la presente serie de casos presentamos la experiencia de un centro oncológico en el abordaje de tumores del mediastino y sus resultados.  Métodos: En el departamento de Jefatura de Cirugía Oncológica del Instituto Oncológico nacional de Solca-Guayaquil, durante los meses de Enero del 2013 a Enero 2017 se realizó un estudio descriptivo, retrospectivo. Se analizaron todos los casos de pacientes derivados del área de pre admisión con diagnóstico inicial de tumor de mediastino, a los cuales previo a realizarles marcadores tumorales, Tomografía de Tórax, y a quienes se les realizó como método diagnóstico y en algunos casos terapéutico con abordaje quirúrgico. Se excluyeron pacientes con neoplasias de origen secundario, con historias clínicas incompletas que imposibilitaron el análisis. Se estudiaron las variables 

In [7]:
df_text_all_abs.sw_text[0]

'Introducción : pesar difícil acceso anatómico tumores mediastino , resección quirúrgica sigue siendo mejor enfoque diagnóstico terapéutico . objetivo presente serie casos presentamos experiencia centro oncológico abordaje tumores mediastino resultados . Métodos : departamento Jefatura Cirugía Oncológica Instituto Oncológico nacional Solca - Guayaquil , meses Enero 2013 Enero 2017 realizó estudio descriptivo , retrospectivo . analizaron casos pacientes derivados área pre admisión diagnóstico inicial tumor mediastino , cuales previo realizarles marcadores tumorales , Tomografía Tórax , realizó método diagnóstico casos terapéutico abordaje quirúrgico . excluyeron pacientes neoplasias origen secundario , historias clínicas incompletas imposibilitaron análisis . estudiaron variables sexo , edad , Tipo Técnica quirúrgica , localización tumor , diagnostico histopatológico mortalidad perioperatoria . análisis estadístico realizado descriptivo . Resultados : evaluaron 22 pacientes diagnóstico 

In [8]:
# Number of distinct docs
len(set(df_text_all_abs.doc_id))

149424

## Load train-asbtracts codes

We load the codes dataset containing the abstracts IDs associated with CIE-D codes present in train corpus (see `CodiEsp_Exploration` notebook):

In [9]:
# Train-abstracts codes
df_codes_d_train_abs = pd.read_table(corpus_path + "train_abstracts_table_valid_codes_D.tsv", sep='\t', header=None)

In [10]:
df_codes_d_train_abs.columns = ["doc_id", "code"]

In [11]:
df_codes_d_train_abs.shape

(148197, 2)

In [12]:
df_codes_d_train_abs.head()

doc_id    code
0  biblio-1008268     f99
1  biblio-1008288  r09.02
2  biblio-1008344     a90
3  biblio-1008711   i49.9
4  biblio-1008711   i48.0

In [13]:
# Number of distinct docs
len(set(df_codes_d_train_abs["doc_id"]))

108029

In [14]:
# Number of distinct codes
len(set(df_codes_d_train_abs["code"]))

615

We select the documents contained both in the pre-processed all-abstracts text dataset and the train-abstracts codes dataframe:

In [15]:
train_abs_v2_30_docs = set(df_text_all_abs["doc_id"]) & set(df_codes_d_train_abs["doc_id"])

In [16]:
len(train_abs_v2_30_docs)

93828

In [17]:
df_text_train_abs_v2_30 = df_text_all_abs[df_text_all_abs["doc_id"].isin(train_abs_v2_30_docs)]

In [18]:
df_text_train_abs_v2_30.shape

(93828, 3)

In [19]:
df_text_train_abs_v2_30.head()

doc_id                                           raw_text  \
0  biblio-1000005  Introducción: A pesar del difícil acceso anató...   
1  biblio-1000026  Introducción: La enterocolitis neutropénica se...   
2  biblio-1000027  Introducción: La presencia de anticuerpos anti...   
3  biblio-1000028  Introducción: El Carcinoma de lengua móvil es ...   
4  biblio-1000029  Introducción: El cáncer de ovario epitelial au...   

                                             sw_text  
0  Introducción : pesar difícil acceso anatómico ...  
1  Introducción : enterocolitis neutropénica defi...  
2  Introducción : presencia anticuerpos anti erit...  
3  Introducción : Carcinoma lengua móvil tumores ...  
4  Introducción : cáncer ovario epitelial aunque ...

In [21]:
%%time
df_text_train_abs_v2_30.to_csv(path_or_buf=corpus_path + "train_abstracts_valid_codes_D_text_raw_sw_v2_30.tsv", 
                               sep="\t", header=True, index=False)

CPU times: user 2.6 s, sys: 160 ms, total: 2.76 s
Wall time: 3.49 s


In [21]:
df_codes_d_train_abs_v2_30 = df_codes_d_train_abs[df_codes_d_train_abs["doc_id"].isin(train_abs_v2_30_docs)]

In [22]:
df_codes_d_train_abs_v2_30.shape

(127531, 2)

In [23]:
df_codes_d_train_abs_v2_30.head()

doc_id    code
0  biblio-1008268     f99
1  biblio-1008288  r09.02
2  biblio-1008344     a90
3  biblio-1008711   i49.9
4  biblio-1008711   i48.0

In [24]:
%%time
df_codes_d_train_abs_v2_30.to_csv(path_or_buf=corpus_path + "train_abstracts_table_valid_codes_D_v2_30.tsv", 
                                                   sep="\t", header=False, index=False)

CPU times: user 112 ms, sys: 0 ns, total: 112 ms
Wall time: 111 ms


As a sanity check, we analyze the BETO sub-token length frequency of train-abstracts texts:

In [25]:
from nltk.corpus import stopwords

spanish_sw = stopwords.words('spanish')

In [26]:
# All Spanish stop words are lowercase
all([w.islower() for w in spanish_sw])

True

In [27]:
from functools import partial

In [28]:
from keras_bert import load_vocabulary, Tokenizer

Using TensorFlow backend.


In [29]:
base_path = "../bert_models/"
vocab_file = "vocab.txt"

In [30]:
beto_path = "BETO_cased/"

In [31]:
beto_token_dict = load_vocabulary(base_path + beto_path + vocab_file)

In [32]:
# 31002 expected
len(beto_token_dict)

31002

In [33]:
beto_tokenizer = Tokenizer(token_dict=beto_token_dict, pad_index=1, cased=True)

In [34]:
%%time
df_text_train_abs_v2_30["raw_BETO"] = [len(beto_tokenizer.tokenize(text)) for text in df_text_train_abs_v2_30.raw_text]
df_text_train_abs_v2_30["sw_BETO"] = [len(beto_tokenizer.tokenize(text)) for text in df_text_train_abs_v2_30.sw_text]

/home_ext/guillermo/miniconda3/envs/nlp_keras/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


CPU times: user 4min 5s, sys: 56.9 ms, total: 4min 5s
Wall time: 4min 5s


/home_ext/guillermo/miniconda3/envs/nlp_keras/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
df_text_train_abs_v2_30.head()

doc_id                                           raw_text  \
0  biblio-1000005  Introducción: A pesar del difícil acceso anató...   
1  biblio-1000026  Introducción: La enterocolitis neutropénica se...   
2  biblio-1000027  Introducción: La presencia de anticuerpos anti...   
3  biblio-1000028  Introducción: El Carcinoma de lengua móvil es ...   
4  biblio-1000029  Introducción: El cáncer de ovario epitelial au...   

                                             sw_text  raw_BETO  sw_BETO  
0  Introducción : pesar difícil acceso anatómico ...       444      330  
1  Introducción : enterocolitis neutropénica defi...       393      293  
2  Introducción : presencia anticuerpos anti erit...       487      349  
3  Introducción : Carcinoma lengua móvil tumores ...       391      264  
4  Introducción : cáncer ovario epitelial aunque ...       405      315

In [36]:
col_names = ["Raw", "SW"]

In [38]:
pd.DataFrame({col_names[0]: df_text_train_abs_v2_30["raw_BETO"].describe(), 
              col_names[1]: df_text_train_abs_v2_30["sw_BETO"].describe()})

Raw            SW
count  93828.000000  93828.000000
mean     260.758665    185.687641
std      128.579126     99.097819
min       30.000000     15.000000
25%      162.000000    111.000000
50%      242.000000    169.000000
75%      349.000000    248.000000
max     1504.000000   1162.000000

As expected, the minimum length is 30 for raw text column.